In [1]:
from natasha import AddressExtractor

In [2]:
import pymorphy2
import nltk
from nltk.corpus import stopwords
import re
import pandas as pd

In [17]:
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer('russian')

In [18]:
morph = pymorphy2.MorphAnalyzer()
stop_words = stopwords.words("russian")

def process_text(text):
    lower = (word.lower() for word in nltk.wordpunct_tokenize(text))
    pattern = re.compile("[а-я+0-9-]+")
    words = pattern.findall(" ".join(lower))
    norm_form = (morph.parse(word)[0].normal_form for word in words)
    tokens = [word for word in text if word not in stop_words]
    return " ".join(tokens)

In [41]:
morph = pymorphy2.MorphAnalyzer()
stop_words = stopwords.words("russian")

def process_text(text):
    tok = text.split(' ')
    pattern = re.compile("[А-я+0-9-]+")
    words = pattern.findall(" ".join(tok))
    #norm_form = (stemmer.stem(word) for word in words)
    tokens = [word for word in words if word not in stop_words]
    return " ".join(tokens)

In [42]:
data = pd.read_csv('data/post_data.csv')
data = data.dropna()

In [43]:
df = data[:1000]

In [44]:
text = '''
На Комсомола улице у дома 10....один уехал в светофор, второй на середине перекрестка.
'''

In [20]:
f = open('street_list_spb.txt', 'r')
l = [line.strip() for line in f]

In [23]:
k = 0
o = []
for i in list(df['text']):
    sample = process_text(i)
    p = []
    for j in l:
        if sample.find(stemmer.stem(j)) > 0:
            p.append(j)
    o.append(p)

In [25]:
d = 0
for i in o:
    if len(i)>0:
        d+=1
print(d)

836


In [26]:
o

[['Софийская', 'Софийский'],
 ['Даля',
  'Авиаконструкторов',
  'Конная',
  'Конный',
  'Новая',
  'Парковая',
  'Парковый'],
 ['Обороны', 'Оборонная'],
 ['Большеохтинский'],
 ['Космонавтов', 'Типанова', 'Новая'],
 ['Прямая', 'Прямой', 'Прямой'],
 ['Зольная'],
 ['Портовая', 'Предпортовая'],
 ['Славы', 'Фрунзе'],
 ['Свободы'],
 [],
 [],
 ['Зайцева', 'Стачек'],
 [],
 ['Индустриальный'],
 [],
 ['Даля', 'Восточный', 'Дальневосточный', 'Дальняя', 'Новая'],
 ['Даля', 'Восточный', 'Дальневосточный', 'Дальняя'],
 [],
 [],
 ['Народная'],
 [],
 ['Косыгина', 'Труда', 'Малая'],
 ['Ланская', 'Омская'],
 ['Выборгская',
  'Выборгская',
  'Дворцовая',
  'Дворцовая',
  'Дворцовый',
  'Дворцовый',
  'Морская',
  'Морской',
  'Невская',
  'Невский',
  'Октябрьская',
  'Октябрьский',
  'Приморская',
  'Приморский',
  'Сестрорецкая'],
 ['Мира', 'Кантемировская', 'Лесная', 'Лесной', 'Новая', 'Средний', 'Средняя'],
 ['Грузовой', 'Заневский', 'Невская', 'Невский', 'Новая', 'Новочеркасский'],
 ['Горская', 'Гор

In [80]:
extractor = AddressExtractor()
sample = process_text(df['text'][8])
matches = extractor(process_text(sample))
for match in matches:
    print(match.fact)
match.fact.parts

Address(parts=[Region(name='Фрунзенского', type='район'), Street(name='Ярослава Гашека', type='улица'), Building(number='4', type=None), Building(number='4', type='корпус')])


[Region(name='Фрунзенского',
        type='район'), Street(name='Ярослава Гашека',
        type='улица'), Building(number='4',
          type=None), Building(number='4',
          type='корпус')]

In [140]:
k = 0
extractor = AddressExtractor()
for i in list(df['text']):
    sample = process_text(i)
    matches = extractor(process_text(sample))
    for match in matches:
        print(match.span, match.fact)
    if len(matches)>0:
        k += 1

[220, 270) Address(parts=[Region(name='Фрунзенского', type='район'), Street(name='Ярослава Гашека', type='улица'), Building(number='4', type=None), Building(number='4', type='корпус')])
[15, 37) Address(parts=[Street(name='Кантемировской', type='улица'), Building(number='9', type=None)])
[20, 45) Address(parts=[Street(name='Будапештской', type='улица'), Building(number='95', type=None), Building(number='1', type='корпус')])
[197, 215) Address(parts=[Street(name='Туристкая', type='улица'), Building(number='26', type=None)])
[127, 153) Address(parts=[Street(name='Петроградской', type='набережная'), Building(number='6', type=None)])
[59, 85) Address(parts=[Street(name='Полюстровском', type='проспект'), Building(number='54', type=None)])
[35, 57) Address(parts=[Street(name='Ленинский', type='проспект'), Building(number='126', type=None)])
[16, 45) Address(parts=[Street(name='Гражданском', type='проспект'), Building(number='83', type='дом')])
[16, 39) Address(parts=[Street(name='Московского

In [141]:
print(k)

122


In [112]:
process_text(text)

'На Комсомола улице дома 10 уехал светофор второй середине перекрестка'

In [124]:
extractor = AddressExtractor()
matches = extractor(process_text(list(df['text'])[0]))
for match in matches:
    print(match.span, match.fact)

In [125]:
len(matches)

0

In [94]:
match.fact.parts

[Street(name='Комсомола',
        type='улица'), Building(number='10',
          type='дом')]

In [183]:
s = match.fact.parts[0].name + ' ' + match.fact.parts[0].type

In [184]:
for i in range(1, len(match.fact.parts)):
    s += ' '  + match.fact.parts[i].type + ' ' + match.fact.parts[i].number

TypeError: can only concatenate str (not "NoneType") to str

In [185]:
s

'Лиговском проспект'